In [175]:
## Business Logic

import math
from Tkinter import *
from ttk import *
import ipywidgets as widgets

RISK = 72000
PROBABILITIES = {
    "prob_of_rain_with_no_data": 0.6667,
    "prob_of_rain_with_data": 0.4204,
    "prob_of_mold": 0.4,
    "prob_of_acid": 0.2
}

def get_value(data, spores, harvest, in_bulk, rain=False, mold=False, sugar=0):
    value = 1000
    if spores:
        if harvest:
            value = 331000
        else:
            if rain:
                value = 811000
            else:
                if sugar == 25:
                    value = 409000
                elif sugar == 20:
                    value = 349000
                else:
                    value = 289000
    else:
        if harvest:
            value = 341000
        else:
            if rain:
                if mold:
                    value = 821000
                else:
                    if in_bulk:
                        value = 128000
                    else:
                        value = 7000
            else:
                if sugar == 25:
                    value = 419000
                elif sugar == 20:
                    value = 359000
                else:
                    value = 299000
    if not data:
        value = value + 1000
    return value

def calc_u_value(value):
    # print value
    # print RISK
    if RISK == 0:
        return value
    elif RISK > 0:
        return 1.0 - math.exp(-1.0 / RISK * value)
    else:
        return 1.0 + math.exp(-1.0 / RISK * value)

def calc_value(u_value):
    if RISK == 0:
        return u_value
    elif RISK > 0:
        return -1.0 * RISK * math.log(1.0 - u_value);
    else:
        return -1.0 * RISK * math.log(u_value - 1.0);

def calculate(data, spores, harvest, in_bulk, rain=None, mold=None, probs=None):
    if not probs:
        probs = PROBABILITIES

    if harvest:
        value = get_value(data, spores, harvest, in_bulk)
        u_value = calc_u_value(value)
        # print ([value, u_value])
        return [value, u_value]

    if rain == None:
        [rain_v, rain_u] = calculate(data, spores, harvest, in_bulk, rain=True, probs=probs)
        [no_rain_v, no_rain_u] = calculate(data, spores, harvest, in_bulk, rain=False, probs=probs)
        prob_of_rain = probs["prob_of_rain_with_data"] if data else probs["prob_of_rain_with_no_data"]
        u_value = prob_of_rain * rain_u + (1.0-prob_of_rain) * no_rain_u
        # print(rain_u)
        # print(no_rain_u)
        # print(u_value)
        value = calc_value(u_value)
        # print(value)
        # print ([value, u_value])
        return [value, u_value]
    elif rain:
        if spores:
            value = get_value(data, spores, harvest, in_bulk, rain=rain)
            u_value = calc_u_value(value)
            # print ([value, u_value])
            return [value, u_value]
        else:
            if mold == None:
                [mold_v, mold_u] = calculate(data, spores, harvest, in_bulk, rain=rain, mold=True, probs=probs)
                [no_vold_v, no_mold_u] = calculate(data, spores, harvest, in_bulk, rain=rain, mold=False, probs=probs)
                prob_of_mold = probs["prob_of_mold"]
                u_value = prob_of_mold * mold_u + (1.0 - prob_of_mold) * no_mold_u
                value = calc_value(u_value)
                # print ([value, u_value])
                return [value, u_value]
            else:
                value = get_value(data, spores, harvest, in_bulk, rain=rain, mold=mold)
                u_value = calc_u_value(value)
                # print ([value, u_value])
                return [value, u_value]
    else:
        value_25 = get_value(data, spores, harvest, in_bulk, rain=rain, sugar=25)
        value_20 = get_value(data, spores, harvest, in_bulk, rain=rain, sugar=20)
        value_0 = get_value(data, spores, harvest, in_bulk, rain=rain, sugar=0)
        u_value_25 = calc_u_value(value_25)
        u_value_20 = calc_u_value(value_20)
        u_value_0 = calc_u_value(value_0)
        prob_of_acid = probs["prob_of_acid"]
        prob_of_sugar = [(1.0 - prob_of_acid) / 2, (1.0 - prob_of_acid) / 2, prob_of_acid]
        u_value = prob_of_sugar[0] * u_value_25 + prob_of_sugar[1] * u_value_20 + prob_of_sugar[2] * u_value_0
        value = calc_value(u_value)
        # print ([value, u_value])
        return [value, u_value]

def pick_decision(decisions, decision_name, probs):
    def copy_decisions(orig, dest):
        dest.clear()
        for key in orig:
            dest[key] = orig[key]

    try_true_decisions = decisions.copy()
    try_false_decisions = decisions.copy()
    try_true_decisions[decision_name] = True
    try_false_decisions[decision_name] = False
    [v_true, u_true] = find_best_decision(try_true_decisions, probs=probs)
    [v_false, u_false] = find_best_decision(try_false_decisions, probs=probs)
    # if decision_name == "data":
    #     print(decision_name)
    #     print([v_true, u_true])
    #     print([v_false, u_false])
    if u_true > u_false:
        copy_decisions(try_true_decisions, decisions)
        return [v_true, u_true]
    else:
        copy_decisions(try_false_decisions, decisions)
        return [v_false, u_false]

def find_best_decision(decisions, probs=None):
    if decisions["data"] == None:
        return pick_decision(decisions, "data", probs)
    if decisions["spores"] == None:
        return pick_decision(decisions, "spores", probs)
    if decisions["harvest"] == None:
        return pick_decision(decisions, "harvest", probs)
    if decisions["in_bulk"] == None:
        return pick_decision(decisions, "in_bulk", probs)

    # print(decisions)
    [v,u] = calculate(
        decisions["data"],
        decisions["spores"],
        decisions["harvest"],
        decisions["in_bulk"],
        probs=probs)
    # print([v,u])
    return calculate(
        decisions["data"],
        decisions["spores"],
        decisions["harvest"],
        decisions["in_bulk"],
        probs=probs)

def find_values(probs=None):
    [v_data_spores, u_data_spores] = pick_decision(
        {"data": True, "spores": True, "harvest": None, "in_bulk": None}, "harvest", probs)
    [v_data_no_spores, u_data_no_spores] = pick_decision(
        {"data": True, "spores": False, "harvest": None, "in_bulk": None}, "harvest", probs)
    [v_no_data_no_spores, u_no_data_no_spores] = pick_decision(
        {"data": False, "spores": False, "harvest": None, "in_bulk": None}, "harvest", probs)
    [v_no_data_spores, u_no_data_spores] = pick_decision(
        {"data": False, "spores": True, "harvest": None, "in_bulk": None}, "harvest", probs)
    return [
        v_data_no_spores - v_no_data_no_spores,
        v_no_data_spores - v_no_data_no_spores,
        v_data_spores - v_no_data_no_spores
    ]

def format_dollars(value):
    if value >= 0:
        return '${:,.2f}'.format(value)
    else:
        return "$0.00"
#         return '-${:,.2f}'.format(abs(value))
    
def decisions_to_string(decisions):
    string = ""
    if decisions["data"]:
        string += "to buy data, "
    else:
        string += "to not buy data, "

    if decisions["spores"]:
        string += "buy spores, "
    else:
        string += "don't buy spores, "

    if decisions["harvest"]:
        string += "harvest now"
    else:
        string += "harvest later"

    return string

In [176]:
## Calculate Basic Case

decisions = {
            "data": None,
            "spores": None,
            "harvest": None,
            "in_bulk": None
}

[value, u_value] = find_best_decision(decisions)
values = find_values()

widget_recommend = widgets.Label(
    "***We recommend " + decisions_to_string(decisions) + ".***")
widget_value = widgets.Label(
    "This will provide an optimal value of " + format_dollars(value) + ".")
widget_value_of_data = widgets.Label(
    "The value of the data is " + format_dollars(values[0]) + ".")
widget_value_of_spores = widgets.Label(
    "The value of the spores is " + format_dollars(values[1]) + ".")
widget_joint_value = widgets.Label(
    "The joint value is " + format_dollars(values[2]) + ".")

In [177]:
## Basic Case Results

display(widget_recommend)
display(widget_value)
display(widget_value_of_data)
display(widget_value_of_spores)
display(widget_joint_value)



Label(value=u'***We recommend to not buy data, buy spores, harvest later.***')

Label(value=u'This will provide an optimal value of $426,470.34.')

Label(value=u'The value of the data is $0.00.')

Label(value=u'The value of the spores is $84,470.34.')

Label(value=u'The joint value is $43,777.84.')

In [178]:
## User input Scenarios

def update(var=None):
    global decisions
    slider_prob_of_mold.disabled = False
    if choices.value:
        dropdown_data.layout.visibility = "visible"
        dropdown_spores.layout.visibility = "visible"
        if dropdown_spores.value == "Yes":
            slider_prob_of_mold.value = 100.0
            slider_prob_of_mold.disabled = True
    else:
        dropdown_data.layout.visibility = "hidden"
        dropdown_spores.layout.visibility = "hidden"
    
    probs = {
        "prob_of_rain_with_no_data": slider_prob_of_rain.value / 100.0,
        "prob_of_rain_with_data": PROBABILITIES["prob_of_rain_with_data"],
        "prob_of_mold": slider_prob_of_mold.value / 100.0,
        "prob_of_acid": slider_prob_of_acid.value / 100.0
        }
    
    update_decisions = {
        "data": None,
        "spores": None,
        "harvest": None,
        "in_bulk": None}

    if choices.value:
        update_decisions["data"] = True if dropdown_data.value == "Yes" else False
        update_decisions["spores"] = True if dropdown_spores.value == "Yes" else False
    else:
        dropdown_data.value = "Yes" if decisions["data"] else "No"
        dropdown_spores.value = "Yes" if decisions["spores"] else "No"
    
    [value, u_value] = find_best_decision(update_decisions, probs)
    values = find_values(probs)
    
    harvest_recommendation = "harvest now" if update_decisions["harvest"] else "harvest later"
    recommend_string = harvest_recommendation if choices.value else decisions_to_string(update_decisions)
    label_recommend.value = "In the above scenario, we recommend " + recommend_string + "."
    label_value.value = "This will provide a value of " + format_dollars(value) + "."
    label_value_of_data.value = "The value of the data is " + format_dollars(values[0]) + "."
    label_value_of_spores.value = "The value of the spores is " + format_dollars(values[1]) + "."
    label_joint_value.value = "The joint value is " + format_dollars(values[2]) + "."
    
    decisions = update_decisions
    

widget_try_title = widgets.Label("Try out your own scenarios below.")

label_desc_prob_of_rain = widgets.Label("Probability of rain without data:")
slider_prob_of_rain = widgets.FloatSlider(
    value=PROBABILITIES["prob_of_rain_with_no_data"]*100,
    min=0,
    max=100.0,
    step=0.01,
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')
label_percent = widgets.Label("%")
slider_prob_of_rain.observe(update, names="value")

label_desc_prob_of_mold = widgets.Label("Probability of mold if raining:")
slider_prob_of_mold = widgets.FloatSlider(
    value=PROBABILITIES["prob_of_mold"]*100,
    min=0,
    max=100.0,
    step=0.01,
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')
slider_prob_of_mold.observe(update, names="value")

label_desc_prob_of_acid = widgets.Label("Probability of acid if not raining:")
slider_prob_of_acid = widgets.FloatSlider(
    value=PROBABILITIES["prob_of_acid"]*100,
    min=0,
    max=100.0,
    step=0.01,
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')
slider_prob_of_acid.observe(update, names="value")

choices = widgets.Checkbox(
    value=False,
    description="Control Individual Choices",
    disabled=False)
choices.observe(update)

dropdown_data = widgets.Dropdown(
    options=["Yes", "No"],
    value= "Yes" if decisions["data"] else "No",
    description="Buy data",
    disabled=False)
dropdown_data.observe(update)

dropdown_spores = widgets.Dropdown(
    options=["Yes", "No"],
    value="Yes" if decisions["spores"] else "No",
    description="Buy spores",
    disabled=False)
dropdown_spores.observe(update)

label_recommend = widgets.Label(value="In the above scenario, we recommend " + decisions_to_string(decisions) + ".")
label_value = widgets.Label(value=widget_value.value)
label_value_of_data = widgets.Label(value=widget_value_of_data.value)
label_value_of_spores = widgets.Label(value=widget_value_of_spores.value)
label_joint_value = widgets.Label(value=widget_joint_value.value)

def on_reset_button(var):
    choices.value = False
    slider_prob_of_rain.value = PROBABILITIES["prob_of_rain_with_no_data"]*100
    slider_prob_of_mold.value = PROBABILITIES["prob_of_mold"] * 100
    slider_prob_of_acid.value = PROBABILITIES["prob_of_acid"] * 100
    update()

button_reset = widgets.Button(
    description="Reset Scenario",
    disabled=False)
button_reset.on_click(on_reset_button)



In [179]:
## Display User Input Scenarios

display(widget_try_title)
display(widgets.HBox([
    label_desc_prob_of_rain,
    slider_prob_of_rain,
    label_percent]))

display(widgets.HBox([
    label_desc_prob_of_mold,
    slider_prob_of_mold,
    label_percent]))

display(widgets.HBox([
    label_desc_prob_of_acid,
    slider_prob_of_acid,
    label_percent]))

display(choices)
display(dropdown_data)
dropdown_data.layout.visibility = "hidden"
display(dropdown_spores)
dropdown_spores.layout.visibility = "hidden"

display(label_recommend)
display(label_value)
display(label_value_of_data)
display(label_value_of_spores)
display(label_joint_value)

display(button_reset)

update()



Label(value=u'Try out your own scenarios below.')

SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdQcm9iYWJpbGl0eSBvZiByYWluIHdpdGhvdXQgZGF0YTonKSwgRmxvYXRTbGlkZXIodmFsdWU9NjYuNjcsIHN0ZXA9MC4wMSksIExhYmVsKHbigKY=


SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdQcm9iYWJpbGl0eSBvZiBtb2xkIGlmIHJhaW5pbmc6JyksIEZsb2F0U2xpZGVyKHZhbHVlPTQwLjAsIHN0ZXA9MC4wMSksIExhYmVsKHZhbHXigKY=


SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdQcm9iYWJpbGl0eSBvZiBhY2lkIGlmIG5vdCByYWluaW5nOicpLCBGbG9hdFNsaWRlcih2YWx1ZT0yMC4wLCBzdGVwPTAuMDEpLCBMYWJlbCjigKY=


Checkbox(value=False, description=u'Control Individual Choices')

...

Label(value=u'This will provide an optimal value of $426,470.34.')

Label(value=u'The value of the data is $0.00.')

Label(value=u'The value of the spores is $84,470.34.')

Label(value=u'The joint value is $43,777.84.')

Button(description=u'Reset Scenario', style=ButtonStyle())